In [164]:
import os
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
from tqdm import tqdm
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from skimage import transform
import pickle

In [165]:
def generate_tabular_dataset(srcpath,name):

    exercices = os.listdir(srcpath)


    if not os.path.exists(srcpath + '/' + name + '.csv') :
        data_train = pd.DataFrame(columns = ['NOISE_X', 
                                             'NOISE_Y',
                                             'LEFT_SHOULDER_X',
                                             'LEFT_SHOULDER_Y',
                                             'RIGHT_SHOULDER_X',
                                             'RIGHT_SHOULDER_Y',
                                             'LEFT_ELBOW_X',
                                             'LEFT_ELBOW_Y',
                                             'RIGHT_ELBOW_X',
                                             'RIGHT_ELBOW_Y',
                                             'LEFT_WRIST_X',
                                             'LEFT_WRIST_Y',
                                             'RIGHT_WRIST_X',
                                             'RIGHT_WRIST_Y',
                                             'LEFT_PINKY_X',
                                             'LEFT_PINKY_Y',
                                             'RIGHT_PINKY_X',
                                             'RIGHT_PINKY_Y',
                                             'LEFT_INDEX_X',
                                             'LEFT_INDEX_Y',
                                             'RIGHT_INDEX_X',
                                             'RIGHT_INDEX_Y',
                                             'LEFT_THUMB_X',
                                             'LEFT_THUMB_Y',
                                             'RIGHT_THUMB_X',
                                             'RIGHT_THUMB_Y',
                                             'LEFT_HIP_X',
                                             'LEFT_HIP_Y',
                                             'RIGHT_HIP_X',
                                             'RIGHT_HIP_Y',
                                             'LEFT_KNEE_X',
                                             'LEFT_KNEE_Y',
                                             'RIGHT_KNEE_X',
                                             'RIGHT_KNEE_Y',
                                             'LEFT_ANKLE_X',
                                             'LEFT_ANKLE_Y',
                                             'RIGHT_ANKLE_X',
                                             'RIGHT_ANKLE_Y',
                                             'LEFT_HEEL_X', 
                                             'LEFT_HEEL_Y',
                                             'RIGHT_HEEL_X', 
                                             'RIGHT_HEEL_Y',
                                             'LEFT_FOOT_INDEX_X', 
                                             'LEFT_FOOT_INDEX_Y',
                                             'RIGHT_FOOT_INDEX_X', 
                                             'RIGHT_FOOT_INDEX_Y',
                                             'PATH',
                                             'EXERCICE'])

    else : 
        data_train = pd.read_csv(srcpath + '/' + name + '.csv')

    
    for ex in exercices : 
        if 'csv' not in ex :
            images = os.listdir(srcpath + '/' + ex)
            print('Processing exercice ' + ex)
            if len(images):
                with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:

                    for im in tqdm(images) :
                        path = srcpath + '/' + ex + '/' + im

                        try :
                        
                            if path not in data_train.PATH.unique() :

                                image = cv2.imread(path)
                                image.flags.writeable = False
                                flip_image = tf.image.flip_left_right(image).numpy()
                                to_process = {path : image,
                                              path + '_fliped' : flip_image}


                                for key in to_process.keys():

                                    # Make detection
                                    results = pose.process(to_process[key])
                                    landmarks = results.pose_landmarks.landmark


                                    new_entry = {'NOISE_X' : landmarks[mp_pose.PoseLandmark.NOSE.value].x, 
                                                'NOISE_Y' : landmarks[mp_pose.PoseLandmark.NOSE.value].y,
                                                'LEFT_SHOULDER_X' : landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                                                'LEFT_SHOULDER_Y' : landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
                                                'RIGHT_SHOULDER_X' : landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                                                'RIGHT_SHOULDER_Y' : landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y,
                                                'LEFT_ELBOW_X' : landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                                                'LEFT_ELBOW_Y' : landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y,
                                                'RIGHT_ELBOW_X' : landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                                                'RIGHT_ELBOW_Y': landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y,
                                                'LEFT_WRIST_X' : landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                                                'LEFT_WRIST_Y' : landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y,
                                                'RIGHT_WRIST_X' : landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                                                'RIGHT_WRIST_Y' : landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y,
                                                'LEFT_PINKY_X' : landmarks[mp_pose.PoseLandmark.LEFT_PINKY.value].x,
                                                'LEFT_PINKY_Y' : landmarks[mp_pose.PoseLandmark.LEFT_PINKY.value].y,
                                                'RIGHT_PINKY_X' : landmarks[mp_pose.PoseLandmark.RIGHT_PINKY.value].x,
                                                'RIGHT_PINKY_Y' : landmarks[mp_pose.PoseLandmark.RIGHT_PINKY.value].y,
                                                'LEFT_INDEX_X' : landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].x,
                                                'LEFT_INDEX_Y': landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].y,
                                                'RIGHT_INDEX_X' : landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].x,
                                                'RIGHT_INDEX_Y' : landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].y,
                                                'LEFT_THUMB_X' : landmarks[mp_pose.PoseLandmark.LEFT_THUMB.value].x,
                                                'LEFT_THUMB_Y' : landmarks[mp_pose.PoseLandmark.LEFT_THUMB.value].y,
                                                'RIGHT_THUMB_X' : landmarks[mp_pose.PoseLandmark.RIGHT_THUMB.value].x,
                                                'RIGHT_THUMB_Y' : landmarks[mp_pose.PoseLandmark.RIGHT_THUMB.value].y,
                                                'LEFT_HIP_X' : landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                                                'LEFT_HIP_Y' : landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y,
                                                'RIGHT_HIP_X' : landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                                                'RIGHT_HIP_Y' : landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y,
                                                'LEFT_KNEE_X' : landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                                                'LEFT_KNEE_Y' : landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y,
                                                'RIGHT_KNEE_X' : landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                                                'RIGHT_KNEE_Y' : landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y,
                                                'LEFT_ANKLE_X' : landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                                                'LEFT_ANKLE_Y' : landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y,
                                                'RIGHT_ANKLE_X' : landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                                                'RIGHT_ANKLE_Y' : landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y,
                                                'LEFT_HEEL_X' : landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].x,
                                                'LEFT_HEEL_Y' : landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y,
                                                'RIGHT_HEEL_X' : landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].x, 
                                                'RIGHT_HEEL_Y' : landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].y,
                                                'LEFT_FOOT_INDEX_X' : landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x,  
                                                'LEFT_FOOT_INDEX_Y' : landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y,  
                                                'RIGHT_FOOT_INDEX_X' : landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x,
                                                'RIGHT_FOOT_INDEX_Y' : landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y,
                                                'PATH' : key,
                                                'EXERCICE' : ex}

                                    data_train = data_train.append(new_entry,ignore_index=True)
                        except :
                            pass

    data_train.to_csv(srcpath + '/' + name + '.csv',index=False)
    return data_train




In [167]:
df_train = generate_tabular_dataset('./Data Train','train')

Processing exercice pullup


100%|██████████| 612/612 [00:38<00:00, 15.82it/s]


Processing exercice pushup


100%|██████████| 683/683 [00:41<00:00, 16.47it/s]


Processing exercice situp


100%|██████████| 959/959 [01:01<00:00, 15.56it/s]


In [168]:
df_test = generate_tabular_dataset('./Data Test','test')

Processing exercice pullup


100%|██████████| 301/301 [00:16<00:00, 18.08it/s]


Processing exercice pushup


100%|██████████| 303/303 [00:21<00:00, 14.24it/s]


Processing exercice situp


100%|██████████| 521/521 [00:33<00:00, 15.60it/s]


In [169]:
X_cols = ['NOISE_X', 'NOISE_Y', 'LEFT_SHOULDER_X', 'LEFT_SHOULDER_Y',
       'RIGHT_SHOULDER_X', 'RIGHT_SHOULDER_Y', 'LEFT_ELBOW_X', 'LEFT_ELBOW_Y',
       'RIGHT_ELBOW_X', 'RIGHT_ELBOW_Y', 'LEFT_WRIST_X', 'LEFT_WRIST_Y',
       'RIGHT_WRIST_X', 'RIGHT_WRIST_Y', 'LEFT_PINKY_X', 'LEFT_PINKY_Y',
       'RIGHT_PINKY_X', 'RIGHT_PINKY_Y', 'LEFT_INDEX_X', 'LEFT_INDEX_Y',
       'RIGHT_INDEX_X', 'RIGHT_INDEX_Y', 'LEFT_THUMB_X', 'LEFT_THUMB_Y',
       'RIGHT_THUMB_X', 'RIGHT_THUMB_Y', 'LEFT_HIP_X', 'LEFT_HIP_Y',
       'RIGHT_HIP_X', 'RIGHT_HIP_Y', 'LEFT_KNEE_X', 'LEFT_KNEE_Y',
       'RIGHT_KNEE_X', 'RIGHT_KNEE_Y', 'LEFT_ANKLE_X', 'LEFT_ANKLE_Y',
       'RIGHT_ANKLE_X', 'RIGHT_ANKLE_Y', 'LEFT_HEEL_X', 'LEFT_HEEL_Y',
       'RIGHT_HEEL_X', 'RIGHT_HEEL_Y', 'LEFT_FOOT_INDEX_X',
       'LEFT_FOOT_INDEX_Y', 'RIGHT_FOOT_INDEX_X', 'RIGHT_FOOT_INDEX_Y']
       
target = 'EXERCICE'
encode = LabelEncoder()

In [170]:
X_train = df_train[X_cols].apply(lambda x : x - np.mean(x),axis=1)
y_train = encode.fit_transform(df_train[target])

In [171]:
X_test = df_test[X_cols].apply(lambda x : x - np.mean(x),axis=1)
y_test = encode.transform(df_test[target])

In [172]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(64,activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(32,activation = 'relu'))
model.add(tf.keras.layers.Dense(3,activation='softmax'))

In [173]:
model.compile(loss = 'SparseCategoricalCrossentropy')

In [174]:
model.fit(X_train,y_train,epochs = 200,validation_data= (X_test,y_test))

Epoch 1/200
97/97 [==============================] - 1s 4ms/step - loss: 0.8866 - val_loss: 0.6794
Epoch 2/200
97/97 [==============================] - 0s 2ms/step - loss: 0.6321 - val_loss: 0.5262
Epoch 3/200
97/97 [==============================] - 0s 2ms/step - loss: 0.5438 - val_loss: 0.4620
Epoch 4/200
97/97 [==============================] - 0s 2ms/step - loss: 0.4778 - val_loss: 0.4109
Epoch 5/200
97/97 [==============================] - 0s 2ms/step - loss: 0.4542 - val_loss: 0.3956
Epoch 6/200
97/97 [==============================] - 0s 2ms/step - loss: 0.4329 - val_loss: 0.3583
Epoch 7/200
97/97 [==============================] - 0s 2ms/step - loss: 0.4267 - val_loss: 0.3468
Epoch 8/200
97/97 [==============================] - 0s 2ms/step - loss: 0.3969 - val_loss: 0.3330
Epoch 9/200
97/97 [==============================] - 0s 2ms/step - loss: 0.4003 - val_loss: 0.3326
Epoch 10/200
97/97 [==============================] - 0s 2ms/step - loss: 0.3952 - val_loss: 0.3257
Epoch 11/

In [175]:
proba_model = model.predict(X_test)
predictions = np.argmax(proba_model,axis=1)
metrics = tf.keras.metrics.Accuracy()

In [176]:
print('Accuracy : {}'.format(metrics(y_test,predictions)))

Accuracy : 0.9405617117881775


In [177]:
with open('./Model/encoder.pkl','wb') as file:
    pickle.dump(encode,file)

In [178]:
model.save('./Model/model')

INFO:tensorflow:Assets written to: ./Model/model\assets


In [179]:
X_train

,NOISE_X,NOISE_Y,LEFT_SHOULDER_X,LEFT_SHOULDER_Y,RIGHT_SHOULDER_X,RIGHT_SHOULDER_Y,LEFT_ELBOW_X,LEFT_ELBOW_Y,RIGHT_ELBOW_X,RIGHT_ELBOW_Y,...,RIGHT_ANKLE_X,RIGHT_ANKLE_Y,LEFT_HEEL_X,LEFT_HEEL_Y,RIGHT_HEEL_X,RIGHT_HEEL_Y,LEFT_FOOT_INDEX_X,LEFT_FOOT_INDEX_Y,RIGHT_FOOT_INDEX_X,RIGHT_FOOT_INDEX_Y
0,0.059655,0.146964,0.019389,0.085886,0.110014,0.084151,0.010225,0.211571,0.137565,0.210791,...,0.180234,-0.542457,-0.052459,-0.572690,0.173818,-0.558042,-0.085460,-0.583785,0.175312,-0.580455
1,0.126713,0.154800,0.071201,0.067161,0.176918,0.061373,0.043105,0.006470,0.188866,0.013856,...,0.213507,-0.437524,0.005811,-0.423736,0.197957,-0.460288,-0.032571,-0.457869,0.260592,-0.526465
2,0.013851,-0.126850,0.048552,-0.068604,-0.022612,-0.062302,0.064936,-0.151960,-0.035816,-0.146516,...,-0.034327,0.269280,0.026405,0.297228,-0.023487,0.267396,0.019411,0.324483,-0.036927,0.313910
3,0.025829,-0.168910,0.042444,-0.109269,0.019472,-0.103960,0.031089,-0.159862,0.001456,-0.165782,...,-0.007913,0.253104,-0.003496,0.248033,-0.007121,0.254263,-0.015090,0.267554,-0.017327,0.279987
4,-0.025182,-0.220927,-0.026732,-0.135479,-0.016627,-0.132162,-0.024361,-0.179491,-0.013022,-0.185621,...,-0.039470,0.423227,-0.020570,0.456239,-0.032976,0.447588,-0.048008,0.467762,-0.070141,0.460726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3087,-0.014262,-0.031979,-0.158272,0.189914,0.122904,0.218721,-0.307927,0.255882,0.217329,0.237005,...,-0.008301,-0.252513,-0.075364,-0.268421,-0.024349,-0.220754,-0.073054,-0.383514,-0.048712,-0.323854
3088,-0.047457,-0.021628,-0.188214,0.171701,0.091823,0.214419,-0.311179,0.230552,0.203747,0.221326,...,0.035424,-0.281315,-0.053050,-0.179948,-0.002493,-0.236104,-0.011399,-0.256288,-0.043434,-0.317515
3089,-0.024854,0.009457,-0.173477,0.210591,0.106834,0.240004,-0.306835,0.229868,0.205535,0.226209,...,-0.021732,-0.194997,-0.092743,-0.175948,-0.039884,-0.160208,-0.077433,-0.242922,-0.076929,-0.213864
3090,-0.058747,0.023991,-0.182079,0.184939,0.087930,0.232544,-0.279061,0.203701,0.201429,0.199585,...,0.019222,-0.236978,-0.053823,-0.158335,0.002688,-0.205713,-0.002739,-0.220655,-0.023191,-0.270021
